# Creating our HH!
## Step 1: Data 🗃️!
We are going to split this step into multiple sub-steps 📄:
1. ⛏️**Web Scraping**.
3. 🐼**Data Transformation**L.
4. 🧹**Data cleansing**.
5. 📤**Export the Data**.
---

# 🤔 Pre-coding
Lets define and importa some useful stuff before we start coding

In [2]:
# Important imports
import requests as req # Library for HTTP requests (allows you to send HTTP requests etremely easily): https://pypi.org/project/requests/
from bs4 import BeautifulSoup # Python Library for pulling data out of HTML files (or in this case, web page): https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import pandas as pd # For Data Analysis and Manipulation in Python: https://pandas.pydata.org/
import numpy as np # For matricial and array operations
import unicodedata # For normalizing in function below

In [3]:
# Normalize text function
def normalize_text(word):
    """
    This function takes a string 'word' and normalized
    Example: hElLó wÓrlD = HELLO WORLD
    """
    word = str(word) # Making sure this is a string
    upper_word = word.upper() # Only upper case letters
    striped_word = upper_word.strip() # No spaces at the beginning or end of the word
    # No accents -> https://docs.python.org/3/library/unicodedata.html#unicodedata.normalize
    normalized_word = ''.join([
        letter for letter in unicodedata.normalize('NFD', striped_word)
        if unicodedata.category(letter) != 'Mn'
    ])

    return normalized_word

# ⛏️Web Scraping 13/01/2022
Is the process of extracting data from a web page, in this case we are going to use my [college page](http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura) 🤗

In [4]:
# Request to the page function
def request_url(url):
    res = req.get(url)
    if res.status_code not in range(200, 300):
        raise Exception("Something wen wrong", res.tatus_code)
    else:
        print(res.status_code, "- Everythin is fine 🔥")
    return res

In [5]:
# Check our request function
url = 'http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura'
res = request_url(url)
content = res.content
print(content[:15])

200 - Everythin is fine 🔥
b'<!DOCTYPE html>'


In [6]:
# We need a beautiful soup object, not html in a string
soup = BeautifulSoup(content, 'html.parser')
# The schedules are in a table, lets bring it (its in the second one)
all_tables = soup.find_all('table')
schedule_table = all_tables[0] # 👈 THIS IS OUR TABLE
#schedule_table

# 🐼**Data Transformation**: We need to transform to pandas objects to make the cleaning

In [7]:
# We need to check how many columns are there in http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura
# So we need each data component in the first row
column_row = schedule_table.find_all('tr')[0] # 1st row
# Every data component
td_with_column_names = column_row.find_all('td')
# Lets have a look to this column names
for td in td_with_column_names:
    print(td.text.replace('\n', ''), end = " | ")
print()
# I am going to change every column name to a better-shorter-normalized column name
column_names = ['index_in_page', 'NAME', 'GROUP', 'DAY/TIME/ROOM1', 'DAY/TIME/ROOM2', 'DAY/TIME/ROOM3', 'DAY/TIME/ROOM4', 'PROFESSORS'] # 👈 THIS IS OUR COLUMN NAMES LIST
for name in column_names:
    print(name, end = ' | ')
    
# Just comparing the number of columns so there is no error
assert len(column_names) == len(td_with_column_names), "Don't have the same lenght! CORRECT IT!"

# | UNIDAD DE APRENDIZAJE  | GRUPO | DÍA/HORA/AULA | DÍA/HORA/AULA | DÍA/HORA/AULA | DÍA/HORA/AULA | PROFESOR | 
index_in_page | NAME | GROUP | DAY/TIME/ROOM1 | DAY/TIME/ROOM2 | DAY/TIME/ROOM3 | DAY/TIME/ROOM4 | PROFESSORS | 

In [8]:
# Now we need to get the actual data IN THE ROWS
all_rows = schedule_table.find_all('tr')
schedules = []
for row in all_rows[1:]: # Starts in 1 because the 0th is the row with the column names
    tds = row.find_all('td') # Need the data of each row
    # We are going to create a dictionary for each row
    d_row = {}
    for index, column in enumerate(column_names):
        d_row[column] = tds[index].text # {column: tds[index]}
        
    # And we need to add it to the schedules list
    schedules.append(d_row)
schedules[23] # 👈 OUR SCHEDULES IN A PSEUDO-JSON FORMAT

{'index_in_page': '24',
 'NAME': 'BIOFÍSICA MÉDICA',
 'GROUP': 'A',
 'DAY/TIME/ROOM1': 'MARTES/10-12/C2',
 'DAY/TIME/ROOM2': 'MIÉRCOLES/10-12/C2',
 'DAY/TIME/ROOM3': '',
 'DAY/TIME/ROOM4': '',
 'PROFESSORS': 'MODESTO ANTONIO SOSA AQUINO/VÍCTOR HUGO HERNÁNDEZ GONZÁLEZ'}

In [9]:
# And now we can create a pandas data frame
raw_schedules_df = pd.DataFrame(schedules, columns = column_names)
# I need to save this file 
raw_schedules_df.to_csv('../data/raw_schedules.csv', index=False)
raw_schedules_df.head(3)

,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS
0,1,ADMINISTRACIÓN Y MANEJO DE PERSONAL,A,SÁBADO/9-12/F6,,,,ALDELMO EMMANUEL ISRAEL REYES PABLO
1,2,ALEMÁN I,A,SÁBADO/8-11/F7,,,,MA. ISABEL DELGADILLO CANO
2,3,ALGEBRA LINEAL,A,LUNES/8-10/AUDITORIO EDIF. G,MIÉRCOLES/8-10/AUDITORIO EDIF. G,,,MIGUEL ÁNGEL VALLEJO HERNÁNDEZ


# 🧹**Data cleansing**: Lets clean the data and correct some errors
I am gonna list every error and thing as a sub-title in the following cells

**Note:** There are some times that the school gives me the schedules in an excel, so there is no need to scrap the data from the web page, so here I am going to bring that file

In [10]:
# Reading csv document (This covers the case where school gives me an excel file before they update their page)
raw_schedules_og_df = pd.read_csv('../data/raw_schedules.csv')
raw_schedules_og_df.head(10)

,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS
0,1,ADMINISTRACIÓN Y MANEJO DE PERSONAL,A,SÁBADO/9-12/F6,NaN,NaN,NaN,ALDELMO EMMANUEL ISRAEL REYES PABLO
1,2,ALEMÁN I,A,SÁBADO/8-11/F7,NaN,NaN,NaN,MA. ISABEL DELGADILLO CANO
2,3,ALGEBRA LINEAL,A,LUNES/8-10/AUDITORIO EDIF. G,MIÉRCOLES/8-10/AUDITORIO EDIF. G,NaN,NaN,MIGUEL ÁNGEL VALLEJO HERNÁNDEZ
3,4,ÁLGEBRA LINEAL,B,MARTES/8-10/F1,JUEVES/8-10/F1,NaN,NaN,TEODORO CÓRDOVA FRAGA
4,5,ÁLGEBRA LINEAL,C,MARTES/10-12/F1,JUEVES/10-12/F1,NaN,NaN,OCTAVIO JOSÉ OBREGÓN DÍAZ
5,6,ÁLGEBRA LINEAL,D,MIÉRCOLES/12-14/F1,VIERNES/12-14/F1,NaN,NaN,ERASTO ORTIZ RICARDO
6,7,ÁLGEBRA LINEAL,E,LUNES/15-17/G1,MIÉRCOLES/15-17/G1,NaN,NaN,JOSÉ DE JESÚS BERNAL ALVARADO
7,8,ANÁLISIS DE CIRCUITOS,A,MARTES/8-10/LAB. DE ELECTRÓNICA EDIF. G,JUEVES/8-12/LAB. DE ELECTRÓNICA EDIF. G,NaN,NaN,CARLOS VILLASEÑOR MORA
8,9,ANÁLISIS DE CIRCUITOS,B,LUNES/12-16/LAB. DE ELECTRÓNICA EDIF.G,MIÉRCOLES/8-10/LAB. DE ELECTRÓNICA EDIF. G,NaN,NaN,JOSÉ MARCO BALLEZA ORDAZ
9,10,ANÁLISIS DE LA CULTURA MEXICANA,A,LUNES/16-19/AUDITORIO DEL EDIF. G,NaN,NaN,NaN,FERNANDO AGUAS ÁNGEL


In [11]:
# Change the columns again because of the case they give me an excel file, remembering I need always this column names:
# index_in_page | NAME | GROUP | DAY/TIME/ROOM1 | DAY/TIME/ROOM2 | DAY/TIME/ROOM3 | DAY/TIME/ROOM4 | PROFESSORS |    

column_names = [
    "index_in_page", 
    "NAME", 
    "GROUP", 
    "DAY/TIME/ROOM1",
    "DAY/TIME/ROOM2",
    "DAY/TIME/ROOM3",
    "DAY/TIME/ROOM4",
    "PROFESSORS",
]

# Just comparing the number of columns so there is no error
assert len(column_names) == len(raw_schedules_og_df.columns), f"Don't have the same lenght!"

In [12]:
# Create the replace dictionary for the rename function for pandas
column_replace_dict = {og_col: new_col for og_col, new_col in zip(raw_schedules_og_df.columns, column_names)}
# Replace the column names
raw_schedules_df = raw_schedules_og_df.rename(columns = column_replace_dict)
raw_schedules_df.head(10)

,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS
0,1,ADMINISTRACIÓN Y MANEJO DE PERSONAL,A,SÁBADO/9-12/F6,NaN,NaN,NaN,ALDELMO EMMANUEL ISRAEL REYES PABLO
1,2,ALEMÁN I,A,SÁBADO/8-11/F7,NaN,NaN,NaN,MA. ISABEL DELGADILLO CANO
2,3,ALGEBRA LINEAL,A,LUNES/8-10/AUDITORIO EDIF. G,MIÉRCOLES/8-10/AUDITORIO EDIF. G,NaN,NaN,MIGUEL ÁNGEL VALLEJO HERNÁNDEZ
3,4,ÁLGEBRA LINEAL,B,MARTES/8-10/F1,JUEVES/8-10/F1,NaN,NaN,TEODORO CÓRDOVA FRAGA
4,5,ÁLGEBRA LINEAL,C,MARTES/10-12/F1,JUEVES/10-12/F1,NaN,NaN,OCTAVIO JOSÉ OBREGÓN DÍAZ
5,6,ÁLGEBRA LINEAL,D,MIÉRCOLES/12-14/F1,VIERNES/12-14/F1,NaN,NaN,ERASTO ORTIZ RICARDO
6,7,ÁLGEBRA LINEAL,E,LUNES/15-17/G1,MIÉRCOLES/15-17/G1,NaN,NaN,JOSÉ DE JESÚS BERNAL ALVARADO
7,8,ANÁLISIS DE CIRCUITOS,A,MARTES/8-10/LAB. DE ELECTRÓNICA EDIF. G,JUEVES/8-12/LAB. DE ELECTRÓNICA EDIF. G,NaN,NaN,CARLOS VILLASEÑOR MORA
8,9,ANÁLISIS DE CIRCUITOS,B,LUNES/12-16/LAB. DE ELECTRÓNICA EDIF.G,MIÉRCOLES/8-10/LAB. DE ELECTRÓNICA EDIF. G,NaN,NaN,JOSÉ MARCO BALLEZA ORDAZ
9,10,ANÁLISIS DE LA CULTURA MEXICANA,A,LUNES/16-19/AUDITORIO DEL EDIF. G,NaN,NaN,NaN,FERNANDO AGUAS ÁNGEL


## ❌ 1. Accents and upper letters
We need to normalize every field (we could do this before in the data extract part, but I think is better to separate things correctly)

In [13]:
# Function for normalizing the df with help of our function in 🤔 Pre-coding part: "normalize_text"
def normalize_df(column):
    normalized_column = [] # A list for the normalized column
    for cell in column:
        normalized_column.append(normalize_text(cell))

    return normalized_column

In [14]:
normalized_data = raw_schedules_df.apply(normalize_df)
normalized_data.head(10) # 👈 OUR DATA NORMALIZED

,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS
0,1,ADMINISTRACION Y MANEJO DE PERSONAL,A,SABADO/9-12/F6,NAN,NAN,NAN,ALDELMO EMMANUEL ISRAEL REYES PABLO
1,2,ALEMAN I,A,SABADO/8-11/F7,NAN,NAN,NAN,MA. ISABEL DELGADILLO CANO
2,3,ALGEBRA LINEAL,A,LUNES/8-10/AUDITORIO EDIF. G,MIERCOLES/8-10/AUDITORIO EDIF. G,NAN,NAN,MIGUEL ANGEL VALLEJO HERNANDEZ
3,4,ALGEBRA LINEAL,B,MARTES/8-10/F1,JUEVES/8-10/F1,NAN,NAN,TEODORO CORDOVA FRAGA
4,5,ALGEBRA LINEAL,C,MARTES/10-12/F1,JUEVES/10-12/F1,NAN,NAN,OCTAVIO JOSE OBREGON DIAZ
5,6,ALGEBRA LINEAL,D,MIERCOLES/12-14/F1,VIERNES/12-14/F1,NAN,NAN,ERASTO ORTIZ RICARDO
6,7,ALGEBRA LINEAL,E,LUNES/15-17/G1,MIERCOLES/15-17/G1,NAN,NAN,JOSE DE JESUS BERNAL ALVARADO
7,8,ANALISIS DE CIRCUITOS,A,MARTES/8-10/LAB. DE ELECTRONICA EDIF. G,JUEVES/8-12/LAB. DE ELECTRONICA EDIF. G,NAN,NAN,CARLOS VILLASENOR MORA
8,9,ANALISIS DE CIRCUITOS,B,LUNES/12-16/LAB. DE ELECTRONICA EDIF.G,MIERCOLES/8-10/LAB. DE ELECTRONICA EDIF. G,NAN,NAN,JOSE MARCO BALLEZA ORDAZ
9,10,ANALISIS DE LA CULTURA MEXICANA,A,LUNES/16-19/AUDITORIO DEL EDIF. G,NAN,NAN,NAN,FERNANDO AGUAS ANGEL


## ❌ 2. Professors Together
There are some subject with more than 1 professor (because of labs or something else), so we need to generate a field for each professor. When this happens, every profressor is separated with a '/' character, so we can use that for creating the new fields

In [15]:
# Grab the professor list
professor_list = normalized_data['PROFESSORS'].to_list()
# Split all this professors in a matrix
professor_matrix = [professor.split('/') for professor in professor_list]
# We need to re-normalize this matrix because it can be there are some not-wanted spaces
for i in range(len(professor_matrix)):
    for j in range(len(professor_matrix[i])):
        professor_matrix[i][j] = normalize_text(professor_matrix[i][j])
# As an example, lets see some multiple professors
professor_matrix[20:22] # One subject with 1 professor and the next one with 3 professors

[['MODESTO ANTONIO SOSA AQUINO',
  'ARTURO GONZALEZ VEGA',
  'TEODORO CORDOVA FRAGA',
  'FRANCISCO MIGUEL VARGAS LUNA'],
 ['TEODORO CORDOVA FRAGA']]

In [16]:
# We need to know how many create so lets calculate it
max_professors = 0
for professors in professor_matrix:
    max_professors = max(len(professors), max_professors)
    
print("The max of professors per subject is:", max_professors)

The max of professors per subject is: 4


In [17]:
# And we need to add the missing columns as blank to professor matrix so its not sparse
for i in range(len(professor_matrix)):
    for missing in range(max_professors - len(professor_matrix[i])):
        professor_matrix[i].append('')
        
professor_matrix[20:22]

[['MODESTO ANTONIO SOSA AQUINO',
  'ARTURO GONZALEZ VEGA',
  'TEODORO CORDOVA FRAGA',
  'FRANCISCO MIGUEL VARGAS LUNA'],
 ['TEODORO CORDOVA FRAGA', '', '', '']]

In [18]:
# So new fields are:
professor_columns = [f"PROFESSOR{index + 1}" for index in range(max_professors)]
professor_columns

['PROFESSOR1', 'PROFESSOR2', 'PROFESSOR3', 'PROFESSOR4']

In [19]:
# Lets create this fields in our df
normalized_data[professor_columns] = professor_matrix

In [20]:
normalized_data[20:22] # Lets check some records in the df # 👈 THIS IS OUR CORRECT DF FOR CLEANSING

,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS,PROFESSOR1,PROFESSOR2,PROFESSOR3,PROFESSOR4
20,21,BASES FISICAS PARA EL DIAGNOSTICO POR IMAGENES,A,MARTES/15-18/C2,VIERNES/12-15/C3,NAN,NAN,MODESTO ANTONIO SOSA AQUINO/ARTURO GONZALEZ VE...,MODESTO ANTONIO SOSA AQUINO,ARTURO GONZALEZ VEGA,TEODORO CORDOVA FRAGA,FRANCISCO MIGUEL VARGAS LUNA
21,22,BIOESTADISTICA,A,LUNES/8-10/F8,VIERNES/8-10/F2,NAN,NAN,TEODORO CORDOVA FRAGA,TEODORO CORDOVA FRAGA,,,


## ❌ 3. Empty cells
If there are empty fields (other than DAY/TIME/ROOM fields because empty cells in this fields is normal) then we need to fix it because that means something is wrong with the algorithm

In [21]:
# First of all, lets standard the spaces and blank spaces to np.nan objects
normalized_data = normalized_data.replace(r'^\s*$', np.nan, regex=True)
normalized_data = normalized_data.replace('NAN', np.nan)

In [22]:
normalized_data.head(3) # 👈 STILL OUR DF

,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS,PROFESSOR1,PROFESSOR2,PROFESSOR3,PROFESSOR4
0,1,ADMINISTRACION Y MANEJO DE PERSONAL,A,SABADO/9-12/F6,NaN,NaN,NaN,ALDELMO EMMANUEL ISRAEL REYES PABLO,ALDELMO EMMANUEL ISRAEL REYES PABLO,NaN,NaN,NaN
1,2,ALEMAN I,A,SABADO/8-11/F7,NaN,NaN,NaN,MA. ISABEL DELGADILLO CANO,MA. ISABEL DELGADILLO CANO,NaN,NaN,NaN
2,3,ALGEBRA LINEAL,A,LUNES/8-10/AUDITORIO EDIF. G,MIERCOLES/8-10/AUDITORIO EDIF. G,NaN,NaN,MIGUEL ANGEL VALLEJO HERNANDEZ,MIGUEL ANGEL VALLEJO HERNANDEZ,NaN,NaN,NaN


In [23]:
# Function to detect if there is an empty cell where it should not be.
def detect_empty_cells(df, columns):
    # Save indices in a list 👇
    empty_cells_index = []
    # Loop for know if it is empty
    for index in range(len(df)):
        for column in columns:
            # print(type(df.at[index, column]), df.at[index, column])
            if pd.isna(df.at[index, column]) or df.at[index, column] == 'NAN' or df.at[index, column] == '':
                empty_cells_index.append([index, column])
                
    return empty_cells_index

In [24]:
# Probemos esta función
columns_to_check = ['NAME', 'GROUP', 'DAY/TIME/ROOM1', 'PROFESSORS', 'PROFESSOR1'] # We only care if these fields are empty becuase that means something is wrong
empty_rows = detect_empty_cells(normalized_data, columns_to_check)

assert not empty_rows, f"It shouldn't be an empty cell in here 👉 {empty_rows}!" # If there are empty rows we need to compare with the link http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura
# If this empty data comes from the data source (the page or excel file) we cannot do anything to correct it, it is school obligation

In [25]:
# Only when we are sure all this records are empty because of the page (and not because of our fault) we can correct them put something like "PENDIENTE"
row = 0
while row < len(empty_rows):
    print(empty_rows[row][0] + 1, empty_rows[row][1])
    if empty_rows[row][1] == 'PROFESSORS':
        row += 1
    row += 1

In [27]:
for row in empty_rows:
    normalized_data.at[row[0], row[1]] = 'PENDIENTE'
    
empty_rows2 = detect_empty_cells(normalized_data, columns_to_check)

assert not empty_rows2, f"It shouldn't be an empty cell in here 👉 {empty_rows2}!" # If there are empty rows we need to compare with the link http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura
# If this empty data comes from the data source (the page or excel file) we cannot do anything to correct it, it is school obligation

## ❌ 4. Errors in DAY/TIME/ROOM fields
We need to format this field form, I choose:

_day/start_hour-end_hour/room_

por ejemplo:

_LUNES/14-16/F9_

Lunes is Monday in Spanish and F9 is the class room name.
In the following code we are going to fix the schedules that do not comply with the format 👇.

In [29]:
# Function to detect a bad format in dates (date by date)
def detect_wrong_dates(date):
    # We're not checking NAN values
    if date == 'NAN' or pd.isna(date) or date == 'PENDIENTE':
        return False, None
    # Split for each "/" to obtain every DAY/TIME/ROOM
    date_split = date.split('/')

    # With this we can find 2 possible errors
    # 1. The date field does not have 3 "/"
    # 2. That, the start-end time does not have any "-" 

    # Detect the 1st
    if len(date_split) != 3:
        return True, 'Slash'

    # Detect the 2nd
    hours = date_split[1] # Porque queremos checar la hora
    if hours and len(hours.split('-')) != 2:
        return True, 'Hour'
        
        
    return False, None

In [30]:
# The above 👆 function only works for one date, we need to create a function that runs that 👆 function for all the df
def detect_wrong_dates_in_df(df, date_columns):
    # Iterating over rows
    for index, row in df.iterrows():
        for column in date_columns:
            detection = detect_wrong_dates(row[column])
            if detection[0]:
                print(f'{detection[1]} Error')
                print(f"Index={index}, #Pag={row['index_in_page']}, Column={column[-1]}")

In [31]:
# Test the function
date_columns = [f'DAY/TIME/ROOM{index+1}' for index in range(4)]
detect_wrong_dates_in_df(normalized_data, date_columns) # 😱 Too many errors, there is no option but to fix these by hand ✋
print("------------------------------------------------------")

Slash Error
Index=10, #Pag=11, Column=1
Slash Error
Index=19, #Pag=20, Column=2
Slash Error
Index=28, #Pag=29, Column=4
Slash Error
Index=44, #Pag=45, Column=1
Slash Error
Index=47, #Pag=48, Column=2
Slash Error
Index=143, #Pag=144, Column=2
Slash Error
Index=172, #Pag=173, Column=1
------------------------------------------------------


If we check out the page http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura we can see that the errors are human errors and these errors are no predictable at all, so we need to fix these erros by hand 🥶

### 💨 Fix 10

In [32]:
# 👀 what is happening?
normalized_data.at[10, 'DAY/TIME/ROOM1'] # It doesn't have a slash between the 14 and F2

'MARTES15-17/F8'

In [33]:
# Corrijamos
normalized_data.at[10, 'DAY/TIME/ROOM1'] = "MARTES/15-17/F8" # 'MARTES/15-17/F8'
normalized_data.at[10, 'DAY/TIME/ROOM1']

'MARTES/15-17/F8'

### 💨 Fix 19

In [34]:
# 👀 what is happening?
normalized_data.at[19, 'DAY/TIME/ROOM2']

'VIERNES/15-17F7'

In [35]:
normalized_data.at[19, 'DAY/TIME/ROOM2'] = "VIERNES/15-17/F7"
normalized_data.at[19, 'DAY/TIME/ROOM2']

'VIERNES/15-17/F7'

### 💨 Fix 28

In [36]:
# 👀 what is happening?
normalized_data.at[28, 'DAY/TIME/ROOM4']

'VIERNES/8-11LAB. DE BIOLOGIA EDIF. G'

In [37]:
normalized_data.at[28, 'DAY/TIME/ROOM4'] = 'VIERNES/8-11/LAB. DE BIOLOGIA EDIF. G'
normalized_data.at[28, 'DAY/TIME/ROOM4']

'VIERNES/8-11/LAB. DE BIOLOGIA EDIF. G'

### 💨 Fix 44

In [38]:
# 👀 what is happening?
normalized_data.at[44, 'DAY/TIME/ROOM1']

'MARTES/15-17F2'

In [39]:
normalized_data.at[44, 'DAY/TIME/ROOM1'] = 'MARTES/15-17/F2'
normalized_data.at[44, 'DAY/TIME/ROOM1']

'MARTES/15-17/F2'

### 💨 Fix 47

In [40]:
# 👀 what is happening?
normalized_data.at[47, 'DAY/TIME/ROOM2']

'VIERNES/ 17-19//SALA DE JUNTAS EDIF. B'

In [41]:
normalized_data.at[47, 'DAY/TIME/ROOM2'] = 'VIERNES/17-19/SALA DE JUNTAS EDIF. B'
normalized_data.at[47, 'DAY/TIME/ROOM2']

'VIERNES/17-19/SALA DE JUNTAS EDIF. B'

### 💨 Fix 143

In [43]:
# 👀 what is happening?
normalized_data.at[143, 'DAY/TIME/ROOM2']

'MIERCOLES12-14/C2'

In [44]:
normalized_data.at[143, 'DAY/TIME/ROOM2'] = 'MIERCOLES/12-14/C2'
normalized_data.at[143, 'DAY/TIME/ROOM2']

'MIERCOLES/12-14/C2'

### 💨 Fix 172

In [45]:
# 👀 what is happening?
normalized_data.at[172, 'DAY/TIME/ROOM1']

'JUEVES15-18/AUDITORIO DE EDIF. G'

In [46]:
normalized_data.at[172, 'DAY/TIME/ROOM1'] = 'JUEVES/15-18/AUDITORIO DE EDIF. G'
normalized_data.at[172, 'DAY/TIME/ROOM1']

'JUEVES/15-18/AUDITORIO DE EDIF. G'

In [47]:
# Lets look for errors again in case we didn't fix them all
detect_wrong_dates_in_df(normalized_data, date_columns) # ✨ No more date errors

## 🤔 5. Add useful fields
For making things easier in the algorithm part, I need to add some fields, that are already in the table, but in a different way. Example: Separate DAY/TIME/ROOM in a field called DAY, another field called TIME and another field called ROOM.

And, even thought this gonna be akward, I need to add the index (not the index_in_page) for tracking propuses in the database, I am going to add the _ID column

In [48]:
date_column = [column_name for column_name in normalized_data.columns if 'DAY/' in column_name]

for index, column in enumerate(date_column):
    normalized_data[f"DAY{index + 1}"] = list(map(lambda day: day.split('/')[0].strip() if not pd.isna(day) and day  != 'PENDIENTE' and day else np.nan, normalized_data[column].tolist()))
    normalized_data[f"TIME{index + 1}"] = list(map(lambda time: time.split('/')[1].strip() if not pd.isna(time) and time  != 'PENDIENTE' and time else np.nan, normalized_data[column].tolist()))
    normalized_data[f"ROOM{index + 1}"] = list(map(lambda room: room.split('/')[2].strip() if not pd.isna(room) and room  != 'PENDIENTE' and room else np.nan, normalized_data[column].tolist()))

try:
    normalized_data.insert(0, '_ID', range(0, len(normalized_data)))
except:
    pass

normalized_data.head(3)

,_ID,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS,PROFESSOR1,...,ROOM1,DAY2,TIME2,ROOM2,DAY3,TIME3,ROOM3,DAY4,TIME4,ROOM4
0,0,1,ADMINISTRACION Y MANEJO DE PERSONAL,A,SABADO/9-12/F6,NaN,NaN,NaN,ALDELMO EMMANUEL ISRAEL REYES PABLO,ALDELMO EMMANUEL ISRAEL REYES PABLO,...,F6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,ALEMAN I,A,SABADO/8-11/F7,NaN,NaN,NaN,MA. ISABEL DELGADILLO CANO,MA. ISABEL DELGADILLO CANO,...,F7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,ALGEBRA LINEAL,A,LUNES/8-10/AUDITORIO EDIF. G,MIERCOLES/8-10/AUDITORIO EDIF. G,NaN,NaN,MIGUEL ANGEL VALLEJO HERNANDEZ,MIGUEL ANGEL VALLEJO HERNANDEZ,...,AUDITORIO EDIF. G,MIERCOLES,8-10,AUDITORIO EDIF. G,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# we need to check out if there is any empty cell again
room_cols = ['ROOM1']
empty_rows3 = detect_empty_cells(normalized_data, room_cols)
empty_rows3

[[22, 'ROOM1']]

In [50]:
for row in empty_rows3:
    normalized_data.at[row[0], row[1]] = 'PENDIENTE'
    
empty_rows3 = detect_empty_cells(normalized_data, columns_to_check)

assert not empty_rows3, f"It shouldn't be an empty cell in here 👉 {empty_rows2}!" # If there are empty rows we need to compare with the link http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura
# If this empty data comes from the data source (the page or excel file) we cannot do anything to correct it, it is school obligation

## ❌ 6. Remove the NAN and replace it with a blank space
For the frontend we will need to remove all the NaN data and just to update them to a '' blank space.

In [51]:
#normalized_data = normalized_data.replace(np.nan, '', regex=True)
#normalized_data.head(3)

## ❌ 7. Check the correct data type in each column
We need to be sure that, for example, every professor, name, group and Day field starts with a letter, and every TIME starts and ends with a number.

In [134]:
def check_initial_letter(df, column_name):
    for index, name in enumerate(df[column_name]):
        string_name = str(name).strip()
        # print(index, string_name, column_name)
        if string_name:
            if not (string_name[0].isalpha() and string_name[-1].isalpha() or string_name[-1].isdigit()):
                return f"The record -> {index} in column -> {column_name} does not start or end with a letter -> {name}"
        
    return False
        
def check_initial_number(df, column_name):
    for index, name in enumerate(df[column_name]):
        string_name = str(name).strip()
        if (string_name and string_name.upper() != 'NAN') and not (string_name[0].isdigit() and string_name[-1].isdigit()):
            return f"The record -> {index} in column -> {column_name} does not start or end with a number -> {name}"
        
    return False


In [135]:
letter_columns = [
    'NAME', 
    'GROUP',
    'DAY/TIME/ROOM1',
    'DAY/TIME/ROOM2',
    'DAY/TIME/ROOM3',
    'DAY/TIME/ROOM4',
    'DAY1',
    'DAY2',
    'DAY3',
    'DAY4',
    'PROFESSORS',
    'PROFESSOR1',
    'PROFESSOR2',
    'PROFESSOR3',
    'PROFESSOR4',
    'ROOM1',
    'ROOM2',
    'ROOM3',
    'ROOM4',
]
number_columns = [
    "_ID",
    "TIME1",
    "TIME2",
    "TIME3",
    "TIME4",
    "index_in_page"
]

ignored_columns= [ # We do not need to check all the data because it is probable we are not going to use all the columns
]

missing = [miss for miss in normalized_data.columns if miss not in letter_columns and miss not in number_columns and miss not in ignored_columns]
# Asserting we are checking all the columns
assert len(letter_columns) + len(number_columns) + len(ignored_columns) == len(normalized_data.columns), f"{len(letter_columns) + len(number_columns)} -> {len(normalized_data.columns)}. \nMissing -> {missing}"

# Check letter data
for column in letter_columns:
    error = check_initial_letter(normalized_data, column)
    if error: print(error)
    
# Check number data
for column in number_columns:
    error = check_initial_number(normalized_data, column)
    if error: print(error)

The record -> 22 in column -> DAY/TIME/ROOM1 does not start or end with a letter -> MARTES/8-10/
The record -> 22 in column -> DAY/TIME/ROOM2 does not start or end with a letter -> JUEVES/8-10/


# 📤**Export the Data**
We are done with the data analysis and cleaning, we can now export this file to create the algorithm for create the schedule combinations

In [55]:
# In JSON because that is how the data is going to be loaded from the request to the API
# normalized_data.to_json('../data/clean_schedules.json', orient = 'records')
# Lets import in csv because we are going to use airtable and that is the format used there
normalized_data.to_csv('../data/clean_schedules.csv', index = False)

# 🤔 What is next?
Now that I have the clean data in a CSV format, this file is going to be used to create (or update) a table in Airtable, which I am going to use in the following steps for creating the algorithm for making the schedule combinations.

# 👾 Extra
Comparing past data with current data:

In [67]:
past_data = pd.read_csv('../data/past_clean_schedules.csv')
past_data.set_index(['_ID'], inplace = True)
past_data.head(3)

,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS,PROFESSOR1,PROFESSOR2,...,ROOM1,DAY2,TIME2,ROOM2,DAY3,TIME3,ROOM3,DAY4,TIME4,ROOM4
_ID,,,,,,,,,,,,,,,,,,,,,
0,1,ADMINISTRACION Y MANEJO DE PERSONAL,A,SABADO/9-12/F6,NaN,NaN,NaN,ALDELMO EMMANUEL ISRAEL REYES PABLO,ALDELMO EMMANUEL ISRAEL REYES PABLO,NaN,...,F6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,ALEMAN I,A,SABADO/8-11/F7,NaN,NaN,NaN,MA. ISABEL DELGADILLO CANO,MA. ISABEL DELGADILLO CANO,NaN,...,F7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,ALGEBRA LINEAL,A,LUNES/8-10/AUDITORIO EDIF. G,MIERCOLES/8-10/AUDITORIO EDIF. G,NaN,NaN,MIGUEL ANGEL VALLEJO HERNANDEZ,MIGUEL ANGEL VALLEJO HERNANDEZ,NaN,...,AUDITORIO EDIF. G,MIERCOLES,8-10,AUDITORIO EDIF. G,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
# The length -> Doesn't have to be equal
print('Past Data Length: ', len(past_data))
print('Current Data Length: ', len(normalized_data))

Past Data Length:  194
Current Data Length:  192


In [127]:
# Compare the professorsand schedule
def get_counters_for_check(data):
    subjects = {subject_name: {'PROFESSORS':set(), 'NUMBER': 0} for subject_name in set(data['NAME'])}
    
    for row in data.iterrows():
        for professor in row[1]['PROFESSORS'].split('/'):
            subjects[row[1]['NAME']]['PROFESSORS'].add(professor)
        subjects[row[1]['NAME']]['NUMBER'] += 1
            
    return subjects

In [129]:
past_subjects = get_counters_for_check(past_data)
current_subjects = get_counters_for_check(normalized_data)
# And lets compare the number of elements per subject for each data set
largest_subjects = past_subjects if len(past_subjects) > len(current_subjects) else current_subjects
largest_subjects_name = 'PAST' if len(past_subjects) > len(current_subjects) else 'CURRENT'
smaller_subjects = past_subjects if len(past_subjects) <= len(current_subjects) else current_subjects
smaller_subjects_name = 'PAST' if len(past_subjects) <= len(current_subjects) else 'CURRENT'

# Este for me dice qué materias no están en en el nuevo horario o qué materias están extra en el nuevo
for subject in largest_subjects:
    if subject not in smaller_subjects:
        print(f'{subject} not in {smaller_subjects_name}')
    else:
        if largest_subjects[subject]['NUMBER'] != smaller_subjects[subject]['NUMBER']:
            print(f"{subject} has {largest_subjects[subject]['NUMBER']} appearances on {largest_subjects_name} and {smaller_subjects[subject]['NUMBER']} on {smaller_subjects_name}")
# After reviewing the number of discrepancies we can assure everything is fine comparing with the page: http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura and our database    

INGLES I has 2 appearances on PAST and 1 on CURRENT
INGLES II not in CURRENT


In [133]:
# Check professors
for subject in largest_subjects:
    if subject in smaller_subjects and largest_subjects[subject]['PROFESSORS'] != smaller_subjects[subject]['PROFESSORS']:
        print(f"Professors changed in {subject}: ")
        print(f"{smaller_subjects_name}: {smaller_subjects[subject]['PROFESSORS']}")
        print(f"{largest_subjects_name}: {largest_subjects[subject]['PROFESSORS']}")
        print('-----------------------------------------------------------------------')

Professors changed in PROBABILIDAD Y ESTADISTICA: 
CURRENT: {'MARIO CESAR PEREZ CHAVEZ', 'VICTOR ALFONSO VICENTE BENITEZ', 'MARIA GUADALUPE LEON VERDIN'}
PAST: {'VICTOR ALFONSO VICENTE BENITEZ', 'PENDIENTE', 'MARIA GUADALUPE LEON VERDIN'}
-----------------------------------------------------------------------
Professors changed in INGENIERIA EN REHABILITACION: 
CURRENT: {'PEDRO ULISES MUNOZ GONZALEZ', 'ARTURO VEGA GONZALEZ'}
PAST: {'ARTURO VEGA GONZALEZ'}
-----------------------------------------------------------------------
Professors changed in CALCULO INTEGRAL: 
CURRENT: {'JOSE SOCORRO GARCIA DIAZ', 'ETNA DAFNE YANEZ ROLDAN', 'CARSTEN HOLTFORT', 'MODESTO ANTONIO SOSA AQUINO', 'JOHAN MANUEL ZUNIGA COLLAZOS', 'FRANCISCO SASTRE CARMONA'}
PAST: {'JOSE SOCORRO GARCIA DIAZ', 'ETNA DAFNE YANEZ ROLDAN', 'PENDIENTE', 'MODESTO ANTONIO SOSA AQUINO', 'JOHAN MANUEL ZUNIGA COLLAZOS', 'FRANCISCO SASTRE CARMONA'}
-----------------------------------------------------------------------
Professors ch